In [1]:
# Flask Dashboard for Real-Time Basket Analytics
# ---------------------------------------------
# This Flask app exposes key analytics endpoints and serves a one-page dashboard.

from flask import Flask, jsonify, render_template_string
import sqlite3
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)

DB_PATH = 'baskets.db'

# Helper to load full basket data
def load_data():
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query("""
        SELECT b.id AS basket_id, b.timestamp, i.item
        FROM baskets b
        JOIN basket_items i ON b.id = i.basket_id
        ORDER BY b.timestamp DESC
    """, conn)
    conn.close()
    return df

@app.route("/api/clusters")
def get_clusters():
    try:
        df = load_data()
        if df.empty:
            return jsonify([])

        baskets = df.groupby('basket_id')['item'].apply(list).tolist()
        te = TransactionEncoder()
        X = te.fit_transform(baskets)
        df_matrix = pd.DataFrame(X, columns=te.columns_)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_matrix)
        model = KMeans(n_clusters=3, random_state=42)
        df_matrix['cluster'] = model.fit_predict(X_scaled)

        result = []
        for i in sorted(df_matrix['cluster'].unique()):
            cluster_data = df_matrix[df_matrix['cluster'] == i].drop(columns='cluster')
            top_items = cluster_data.sum().sort_values(ascending=False).head(5)
            result.append({"cluster": int(i), "top_items": top_items.to_dict()})

        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/api/associations")
def get_association_rules():
    try:
        df = load_data()
        if df.empty:
            return jsonify([])

        baskets = df.groupby('basket_id')['item'].apply(list).tolist()
        te = TransactionEncoder()
        X = te.fit_transform(baskets)
        df_encoded = pd.DataFrame(X, columns=te.columns_)

        frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)
        top_rules = rules.sort_values(by='lift', ascending=False).head(10)

        result = []
        for _, row in top_rules.iterrows():
            result.append({
                "lhs": list(row['antecedents']),
                "rhs": list(row['consequents']),
                "support": round(row['support'], 2),
                "confidence": round(row['confidence'], 2),
                "lift": round(row['lift'], 2)
            })
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/api/recent")
def get_recent():
    df = load_data()
    grouped = df.groupby('basket_id').agg({"timestamp": "first", "item": list}).reset_index()
    grouped = grouped.sort_values(by="timestamp").tail(5)
    return jsonify(grouped.to_dict(orient='records'))

@app.route("/api/product_counts")
def get_product_counts():
    df = load_data()
    count_series = df['item'].value_counts().sort_values(ascending=False)
    return jsonify(count_series.to_dict())

@app.route("/api/item_correlation")
def get_item_correlation():
    df = load_data()
    baskets = df.groupby('basket_id')['item'].apply(list).tolist()
    te = TransactionEncoder()
    X = te.fit_transform(baskets)
    df_matrix = pd.DataFrame(X, columns=te.columns_)
    corr = df_matrix.corr().round(2)
    return jsonify(corr.to_dict())

@app.route("/api/sales_over_time")
def get_sales_over_time():
    df = load_data()
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.dropna(subset=['timestamp'])
    df['minute'] = df['timestamp'].dt.floor('min')

    if df.empty:
        return jsonify({'labels': [], 'series': {}})

    sales = df.groupby(['minute', 'item']).size().unstack(fill_value=0).sort_index()
    top_items = sales.sum().sort_values(ascending=False).head(5).index
    sales = sales[top_items]

    return jsonify({
        'labels': sales.index.strftime('%H:%M').tolist(),
        'series': sales.to_dict(orient='list')
    })

@app.route("/")
def index():
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Basket Analytics Dashboard</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            h2 { border: 2px solid #ccc; padding: 10px; background-color: #f9f9f9; }
            section { margin-bottom: 30px; padding: 10px; border: 1px solid #ddd; border-radius: 5px; }
        </style>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<script src="https://cdn.jsdelivr.net/npm/chartjs-chart-matrix@1.1.0/dist/chartjs-chart-matrix.min.js"></script>
        <script>
            let chartInstance = null;
            let lineInstance = null;
            let heatmapInstance = null;

            async function fetchData(endpoint, callback) {
                const res = await fetch(endpoint);
                const data = await res.json();
                callback(data);
            }

            function renderRecent(data) {
                const out = data.map(basket => {
                    return `<li>${basket.timestamp}: [${basket.item.join(', ')}]</li>`;
                }).join('');
                document.getElementById("recent").innerHTML = `<ul>${out}</ul>`;
            }

            function renderBarChart(data) {
                const labels = Object.keys(data);
                const counts = Object.values(data);
                const ctx = document.getElementById('productChart').getContext('2d');
                if (chartInstance) chartInstance.destroy();
                chartInstance = new Chart(ctx, {
                    type: 'bar',
                    data: {
                        labels: labels,
                        datasets: [{
                            label: 'Product Sales Count',
                            data: counts,
                            backgroundColor: 'rgba(75, 192, 192, 0.6)'
                        }]
                    },
                    options: {
                        responsive: true,
                        scales: {
                            x: { ticks: { autoSkip: false } },
                            y: { beginAtZero: true }
                        }
                    }
                });
            }

            function renderClusters(data) {
                const out = data.map(cluster => {
                    const items = Object.entries(cluster.top_items).map(([item, count]) => `<li>${item}: ${count}</li>`).join('');
                    return `<h3>Cluster ${cluster.cluster}</h3><ul>${items}</ul>`;
                }).join('');
                document.getElementById("clusters").innerHTML = out;
            }

            function renderRules(data) {
                const out = data.map(rule => {
                    return `<li><strong>${rule.lhs.join(', ')}</strong> → <strong>${rule.rhs.join(', ')}</strong> (confidence: ${rule.confidence}, lift: ${rule.lift})</li>`;
                }).join('');
                document.getElementById("rules").innerHTML = `<ul>${out}</ul>`;
            }

            function renderLineChart(data) {
                const ctx = document.getElementById('lineChart').getContext('2d');
                const datasets = Object.entries(data.series).map(([label, values]) => ({
                    label: label,
                    data: values,
                    fill: false,
                    borderWidth: 2
                }));
                if (lineInstance) lineInstance.destroy();
                lineInstance = new Chart(ctx, {
                    type: 'line',
                    data: { labels: data.labels, datasets },
                    options: {
                        responsive: true,
                        scales: { y: { beginAtZero: true } }
                    }
                });
            }

            function renderCorrelation(data) {
                const labels = Object.keys(data);
                const matrix = labels.map(row => labels.map(col => data[row][col]));
                const ctx = document.getElementById('heatmapChart').getContext('2d');
                if (heatmapInstance) heatmapInstance.destroy();
                heatmapInstance = new Chart(ctx, {
                    type: 'matrix',
                    data: {
                        labels: labels,
                        datasets: [{
                            label: 'Item Correlation',
                            data: matrix.flatMap((row, i) => row.map((value, j) => ({ x: labels[i], y: labels[j], v: value }))),
                            backgroundColor: ctx => `rgba(0, 123, 255, ${ctx.raw.v})`
                        }]
                    },
                    options: {
                        plugins: { legend: { display: false } },
                        scales: {
                            x: { type: 'category', labels: labels },
                            y: { type: 'category', labels: labels }
                        }
                    }
                });
            }

            function loadDashboard() {
                fetchData(`/api/recent?t=${Date.now()}`, renderRecent);
                fetchData("/api/product_counts", renderBarChart);
                fetchData("/api/clusters", renderClusters);
                fetchData("/api/associations", renderRules);
                fetchData("/api/sales_over_time", renderLineChart);
                fetchData("/api/item_correlation", renderCorrelation);
            }

            window.onload = function() {
                loadDashboard();
                setInterval(loadDashboard, 10000);
            }
        </script>
    </head>
    <body>
        <h1>🛒 Real-Time Basket Analytics Dashboard</h1>

        <section>
            <h2>🆕 Recent Baskets</h2>
            <div id="recent">Loading...</div>
        </section>

        <section>
            <h2>📊 Product Sales Overview</h2>
            <canvas id="productChart" width="600" height="300"></canvas>
        </section>

        <section>
            <h2>📈 Sales Over Time</h2>
            <canvas id="lineChart" width="600" height="300"></canvas>
        </section>

        <section>
            <h2>🔥 Item Correlation Heatmap</h2>
            <canvas id="heatmapChart" width="600" height="600"></canvas>
        </section>

        <section>
            <h2>🔍 Top Items per Cluster</h2>
            <div id="clusters">Loading...</div>
        </section>

        <section>
            <h2>🔗 Association Rules</h2>
            <div id="rules">Loading...</div>
        </section>
    </body>
    </html>
    """
    return render_template_string(html)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/May/2025 11:05:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 11:05:26] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/May/2025 11:05:26] "GET /api/product_counts HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 11:05:26] "GET /api/recent?t=1748509526383 HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 11:05:26] "GET /api/item_correlation HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 11:05:26] "GET /api/sales_over_time HTTP/1.1" 200 -
C:\Users\chris\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
127.0.0.1 - - [29/May/2025 11:05:36] "GET /api/clusters HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 11:05:36] "GET /api/product_counts HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 11:05:36] "GET /api/item